# This code smoothens the x, y centroids positions

## written by Tanvi Deora, based on original version by Mahad Ahmed
## 2020, Sept 15

In [15]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import cv2

In [16]:
# load in all centroids point, 
# transform them back into pixel

In [18]:
centroid = pd.read_csv("./dataFolders/Output/KinematicAnalysis_v5/CentroidAndDistanceData.csv")
centroid.head()

,Unnamed: 0,lightLevel,visitNum,flower,mothID,x,y,name,flower_x,flower_y,cent_x,cent_y,distance
0,0,L50,first,c-3,m9,NaN,NaN,L50_c-3_m9,10.472973,7.871622,NaN,NaN,NaN
1,1,L50,first,c-3,m9,NaN,NaN,L50_c-3_m9,10.472973,7.871622,NaN,NaN,NaN
2,2,L50,first,c-3,m9,NaN,NaN,L50_c-3_m9,10.472973,7.871622,NaN,NaN,NaN
3,3,L50,first,c-3,m9,3.108108,14.256757,L50_c-3_m9,10.472973,7.871622,7.364865,-6.385135,9.747368
4,4,L50,first,c-3,m9,9.222973,15.236486,L50_c-3_m9,10.472973,7.871622,1.250000,-7.364865,7.470190


In [19]:
# get the circle parameters to convert back to pixel dimension

mystr = r"G:/My Drive/Mahad/Light Level Experiments/" \
        r"Data Output from Analysis/motionAnalysis_final/" \
        r"Centroid/circle_parameters.csv"

circle = pd.read_csv(mystr)

In [20]:
def GetCircParams(name):
    r = circle[circle.name == name].circ_radii.values[0]
    x = circle[circle.name == name].circ_x.values[0]
    y = circle[circle.name == name].circ_y.values[0]
    return(r, x, y)

In [21]:
def convertToPixel(x,y,r):
    new_x = x*r/2.5
    new_y = y*r/2.5
    return(new_x, new_y)

In [22]:
from scipy import signal

def smooth(Pixels):
    win_gauss = signal.gaussian(7,3)
    smooth_Pixels = signal.convolve(Pixels, win_gauss, mode='same', method = 'direct') / win_gauss.sum()
    return(smooth_Pixels)

In [23]:
#create a pd with all smoothened x and y centroids in pixel space

mothname = []
visit = []
flower_r = []
flower_x = []
flower_y = []
centroid_x = []
centroid_y = []
p_x = []
p_y = []
s_x = []
s_y = []


for name, df in centroid.groupby('name'):
    x = df.x
    y = df.y
    
    f_r, f_x, f_y = GetCircParams(name)
    # convert back to pixel space
    pixel_x, pixel_y = convertToPixel(x,y,f_r)
    
    # smoothen the trajectory
    smooth_x = smooth(pixel_x)
    smooth_y = smooth(pixel_y)
    
    r = [f_r]*len(smooth_x)
    circ_x = [f_x]*len(smooth_x)
    circ_y = [f_y]*len(smooth_x)
    n = [name]*len(smooth_x)
    v = df.visitNum
    
    mothname.extend(n)
    flower_r.extend(r)
    flower_x.extend(circ_x)
    flower_y.extend(circ_y)
    centroid_x.extend(x)
    centroid_y.extend(y)
    p_x.extend(pixel_x)
    p_y.extend(pixel_y)
    s_x.extend(smooth_x)
    s_y.extend(smooth_y)
    visit.extend(v)
    
uber_df = pd.DataFrame({'name' : mothname,
                        'visit' : visit,
                       'flower_r': flower_r,
                       'flower_x': flower_x,
                       'flower_y': flower_y,
                        'x':centroid_x,
                        'y':centroid_y,
                        'x_pixel': p_x,
                        'y_pixel': p_y,
                        'x_smoothened':s_x,
                        'y_smoothened':s_y,})
uber_df.to_csv('./dataFolders/Output/KinematicAnalysis_v5/SmoothendCentroidInPixelsSpace.csv')